In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, LSTM,
    Dense, Dropout, BatchNormalization, GlobalAveragePooling1D, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import librosa
import scipy.signal as signal
from scipy.signal import resample
import os

# ================= CONFIG =================
FS_TARGET = 2000      # Sampling rate
DURATION = 3.0        # seconds
ANALOG_LEN = 250      # CNN input length
MFCC_LEN = 38         # LSTM input time steps
N_MFCC = 20           # Number of MFCC coefficients

DATA_DIR = r"E:\Final Year Project\Data Training"  # Change to your processed data folder

# ================= LOAD RAW WAV FILES AND PREPROCESS =================
def preprocess_file(file_path):
    # Load audio
    audio, sr = librosa.load(file_path, sr=None)
    if sr != FS_TARGET:
        audio = librosa.resample(audio, sr, FS_TARGET)
    max_len = int(FS_TARGET * DURATION)
    audio = audio[:max_len] if len(audio) > max_len else np.pad(audio, (0, max_len - len(audio)))

    # ---- Analog (CNN) ----
    b, a = signal.butter(4, [20/(FS_TARGET/2), 400/(FS_TARGET/2)], btype='band')
    analog = signal.filtfilt(b, a, audio)
    analog = resample(analog, ANALOG_LEN).reshape(ANALOG_LEN,1)

    # ---- MFCC (LSTM) ----
    mfcc = librosa.feature.mfcc(y=audio, sr=FS_TARGET, n_mfcc=N_MFCC).T
    if mfcc.shape[0] > MFCC_LEN:
        mfcc = mfcc[:MFCC_LEN]
    elif mfcc.shape[0] < MFCC_LEN:
        mfcc = np.pad(mfcc, ((0, MFCC_LEN - mfcc.shape[0]), (0,0)))
    
    return analog, mfcc

# ================= LOAD ALL DATA =================
X_analog_list = []
X_mfcc_list = []
y_list = []

for label_folder in os.listdir(DATA_DIR):
    label_path = os.path.join(DATA_DIR, label_folder)
    if os.path.isdir(label_path):
        label_index = 0 if label_folder.lower() == "normal" else 1
        for file_name in os.listdir(label_path):
            if file_name.lower().endswith(".wav"):
                file_path = os.path.join(label_path, file_name)
                analog, mfcc = preprocess_file(file_path)
                X_analog_list.append(analog)
                X_mfcc_list.append(mfcc)
                y_list.append(label_index)

X_analog = np.array(X_analog_list)
X_mfcc = np.array(X_mfcc_list)
y = np.array(y_list)

# One-hot encode labels
y_cat = to_categorical(y, num_classes=2)

# Train-test split
Xa_tr, Xa_te, Xd_tr, Xd_te, y_tr, y_te = train_test_split(
    X_analog, X_mfcc, y_cat,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# ================= BUILD HYBRID MODEL =================
# CNN Branch
analog_input = Input(shape=(ANALOG_LEN,1))
x = Conv1D(64,5,activation='relu',padding='same')(analog_input)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128,3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = GlobalAveragePooling1D()(x)
x = Dense(64,activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# LSTM Branch
digital_input = Input(shape=(MFCC_LEN,N_MFCC))
y_l = LSTM(64)(digital_input)
y_l = Dense(64,activation='relu')(y_l)
y_l = BatchNormalization()(y_l)

# Fusion
combined = Concatenate()([x,y_l])
z = Dense(64,activation='relu')(combined)
z = Dropout(0.4)(z)
output = Dense(2,activation='softmax')(z)

model = Model([analog_input,digital_input],output)

# ================= COMPILE =================
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ================= TRAIN =================
history = model.fit(
    [Xa_tr, Xd_tr], y_tr,
    epochs=50,
    batch_size=32,
    validation_split=0.15,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True
        )
    ]
)

# ================= EVALUATE =================
pred = model.predict([Xa_te, Xd_te])
pred_cls = np.argmax(pred, axis=1)
true_cls = np.argmax(y_te, axis=1)
print("\nClassification Report\n")
print(classification_report(true_cls, pred_cls))

# ================= SAVE MODEL =================
model.save("hybrid_cnn_lstm_heart_sound_final.h5")
print("\n✅ Hybrid CNN+LSTM model trained and saved successfully!")
